In [ ]:
cd ..

In [ ]:
import os
import sys
sys.path.append('./exp-refgoog/refgoog-dataset/google_refexp_py_lib/')
from refexp import Refexp

import tqdm
import numpy as np

import util.io

In [ ]:
# Specify datasets path.
refexp_filename_trn='./exp-refgoog/refgoog-dataset/google_refexp_dataset_release/google_refexp_train_201511_coco_aligned.json'
refexp_filename_val='./exp-refgoog/refgoog-dataset/google_refexp_dataset_release/google_refexp_val_201511_coco_aligned.json'
coco_filename='./exp-refgoog/coco-dataset/annotations/instances_train2014.json'
image_dir = './exp-refgoog/coco-dataset/images/train2014/'

# Create Refexp instance.
refexp_trn = Refexp(refexp_filename_trn, coco_filename)
refexp_val = Refexp(refexp_filename_val, coco_filename)

In [ ]:
# imdb_format:
#   a list of images
# each image:
#   a dict
#   { "image_path": "...",
#     "regions": [[[x1, y1, x2, y2], (sentence1, sentence2)], ...],
#     "proposals": [[x1, y1, x2, y2], ...],
#     "misc": {"dataset": "coco", ...} }

def build_imdb_from_refexp(refexp, image_dir):
    imdb = []

    imgIds = refexp.getImgIds()
    for n_img_id in tqdm.trange(len(imgIds)):
        img_id = imgIds[n_img_id]
        img = refexp.loadImgs(img_id)[0]
        im_path = os.path.join(image_dir, img['file_name'])
        
        proposals = np.array([_convert_bbox(c['bounding_box']) for c in img['region_candidates']])
        
        regions = _load_refexp_regions(refexp, img_id)
        misc = {"dataset": "coco-G-Ref", "img_id": img_id}
        iminfo = {"im_path": im_path, "regions": regions, "proposals": proposals,
                  "misc": misc}

        imdb.append(iminfo)

    return imdb

def _convert_bbox(bbox_xywh):
    x, y, w, h = bbox_xywh
    return [x, y, x+w-1, y+h-1]

def _load_refexp_regions(refexp, img_id):
    ann_ids_gref = refexp.getAnnIds(img_id)
    anns_gref = refexp.loadAnns(ann_ids_gref)
    ann_ids_coco = refexp.coco.getAnnIds(img_id)
    anns_coco = refexp.coco.loadAnns(ann_ids_coco)
    
    # Align refexp annotations to those in coco
    anns_gref_map = {ann_ids_gref[n]: anns_gref[n] for n in range(len(ann_ids_gref))}
    for n in range(len(ann_ids_coco)):
        if ann_ids_coco[n] in anns_gref_map:
            anns_coco[n]['refexp_ids'] = anns_gref_map[ann_ids_coco[n]]['refexp_ids']
        else:
            anns_coco[n]['refexp_ids'] = []
    
    # Collect all the ground-truth object bounding boxes from COCO
    # Add language annotations to the bounding boxes
    regions = [[_convert_bbox(ann['bbox']),
                _load_refexp_sentences(refexp, ann['refexp_ids']),
                ann['id']] for ann in anns_coco]
    return regions

def _load_refexp_sentences(refexp, refexp_ids):
    exps = [e['raw'] for e in refexp.loadRefexps(refexp_ids)]
    return exps

In [ ]:
imdb_trn = build_imdb_from_refexp(refexp_trn, image_dir)
imdb_val = build_imdb_from_refexp(refexp_val, image_dir)

In [ ]:
if not os.path.isdir('./exp-refgoog/data/imdb'):
    os.mkdir('./exp-refgoog/data/imdb')
    
util.io.save_numpy_obj(imdb_trn, './exp-refgoog/data/imdb/imdb_trn.npy')
util.io.save_numpy_obj(imdb_val, './exp-refgoog/data/imdb/imdb_val.npy')